In [1]:
!pip install -q transformers==4.44.2 datasets sentencepiece evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 74.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 45.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 

In [2]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [3]:
import pandas as pd
import numpy as np
import os, re
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,Seq2SeqTrainingArguments,Seq2SeqTrainer
import evaluate
import nltk
import evaluate
import os
os.environ["WANDB_DISABLED"] = "true"
rouge = evaluate.load("rouge")
nltk.download("punkt")

2025-10-14 09:14:17.581023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760433257.844362      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760433257.921856      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **0. Methodology**

- This project implements abstractive text summarization using the **T5-base** model. It's an encoder-decoder model where attention is used in multiple ways: the encoder uses self-attention to process the input, the decoder uses self-attention on the output, and cross-attention is used for the decoder to attend to the encoder's output. The model is fine-tuned on the **CNN/DailyMail dataset**. For computational efficiency, 20,000 samples from the training set and 2,000 each from validation and test sets were used for fine-tuning and evaluation.
- Evaluation metrics: The ROUGE (Recall-Oriented Understudy for Gisting Evaluation) metric was used for quantitative assessment, including:
    - **ROUGE-1:** Overlap of unigrams (word-level)
    - **ROUGE-2:** Overlap of bigrams
    - **ROUGE-L / ROUGE-Lsum:** Longest common subsequence (measures fluency and structure)
  

In [4]:
train= pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')
train_df = pd.DataFrame(train)
val = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')
val_df = pd.DataFrame(val)
test = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')
test_df = pd.DataFrame(test)
train_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          287113 non-null  object
 1   article     287113 non-null  object
 2   highlights  287113 non-null  object
dtypes: object(3)
memory usage: 6.6+ MB


In [6]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13368 entries, 0 to 13367
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          13368 non-null  object
 1   article     13368 non-null  object
 2   highlights  13368 non-null  object
dtypes: object(3)
memory usage: 313.4+ KB


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11490 entries, 0 to 11489
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          11490 non-null  object
 1   article     11490 non-null  object
 2   highlights  11490 non-null  object
dtypes: object(3)
memory usage: 269.4+ KB


# **1. Data Preprocessing**

In [8]:
#Drop unused columns 
train_df = train_df.drop(columns = ['id'])
val_df = val_df.drop(columns = ['id'])
test_df = test_df.drop(columns = ['id'])

In [9]:
#Check duplicates 
print("Train duplicates: ", train_df.duplicated().sum())
print("Validation duplicates: ", val_df.duplicated().sum())
print("Test duplicates: ", test_df.duplicated().sum())

Train duplicates:  3098
Validation duplicates:  0
Test duplicates:  2


In [10]:
#Drop duplicates 
train_df = train_df.drop_duplicates()
test_df = test_df.drop_duplicates()

In [11]:
#Sampling the dataset for faster training 
training_set = train_df.sample(n=20000, random_state=42)
validation_set = val_df.sample(n=2000, random_state=42)
test_set = test_df.sample(n=2000, random_state=42)
print(f"Train: {training_set.shape}, Val: {validation_set.shape}, Test: {test_set.shape}")

Train: (20000, 2), Val: (2000, 2), Test: (2000, 2)


In [12]:
#Clean text 
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

training_set['article'] = training_set['article'].apply(clean_text)
training_set['highlights'] = training_set['highlights'].apply(clean_text)

validation_set['article'] = validation_set['article'].apply(clean_text)
validation_set['highlights'] = validation_set['highlights'].apply(clean_text)

test_set['article'] = test_set['article'].apply(clean_text)
test_set['highlights'] = test_set['highlights'].apply(clean_text)

In [13]:
# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(training_set)
val_dataset = Dataset.from_pandas(validation_set)
test_dataset = Dataset.from_pandas(test_set)
for ds in [train_dataset, val_dataset, test_dataset]:
    if "__index_level_0__" in ds.column_names:
        ds = ds.remove_columns(["__index_level_0__"])

# **2. Model Training**

In [14]:
#Prepare model 
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [15]:
#Tokenization 
max_input_length = 512
max_target_length = 128

def preprocess (examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_input = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_train = train_dataset.map(preprocess, batched=True)
tokenized_val   = val_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:
tokenized_test = test_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [17]:
#Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [18]:
#Metrics (ROUGE)
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v * 100, 4) for k, v in result.items()}

In [19]:
#Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-summarizer",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,  
    predict_with_generate=True,
    fp16=True,  
    push_to_hub=False,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=500,
)

#Trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [20]:
# Train model
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.625200,1.554892,25.204000,12.546700,21.121100,21.114000,1899.900000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using th

TrainOutput(global_step=2500, training_loss=1.644291845703125, metrics={'train_runtime': 2582.8564, 'train_samples_per_second': 7.743, 'train_steps_per_second': 0.968, 'total_flos': 1.21791578112e+16, 'train_loss': 1.644291845703125, 'epoch': 1.0})

# **3. Evaluation**

In [21]:
#Evaluate on validation set
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 1.5548917055130005, 'eval_rouge1': 25.204, 'eval_rouge2': 12.5467, 'eval_rougeL': 21.1211, 'eval_rougeLsum': 21.114, 'eval_gen_len': 1899.9, 'eval_runtime': 262.0986, 'eval_samples_per_second': 7.631, 'eval_steps_per_second': 0.954, 'epoch': 1.0}


The evaluation results indicate that after one epoch of training, the model achieved a validation loss of 1.55, suggesting it has learned to generate summaries with moderate accuracy but still has room for improvement. The **ROUGE-1** score of **25.20%** and **ROUGE-L** score of **21.12%** show that the model captures some key words and phrases from the reference summaries. 

Notably, the average generated length (eval_gen_len) is around **1900 tokens**, which is far longer than the intended summary length and indicates a configuration issue with generation parameters — the model likely produced excessively long outputs instead of concise summaries.

In [34]:
sample_text = test_df["article"].iloc[0]
inputs = tokenizer("summarize: " + sample_text, return_tensors="pt", max_length=512, truncation=True)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=128, early_stopping=True)

# Print results
print("\nOriginal Article:\n", sample_text[:500], "...")
print("\nGenerated Summary:\n", tokenizer.decode(summary_ids[0], skip_special_tokens=True))



Original Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by t ...

Generated Summary:
 The FAA conducts tests on how quickly passengers can leave a plane, but these tests are conducted using planes with a 31 inch pitch. While most airlines stick to a pitch of 31 inches or above, some airlines fall below this. British Airways has a seat pitch of 31 inches, while easyJet has 29 inches, Thomson's short haul seat pitch is 28 inches and Virgin Atlantic's is 30-31 inches.
